In [1]:
# install qdrant
!pip install qdrant-client 

In [2]:
# initialize client
from qdrant_client import models, QdrantClient

client = QdrantClient(":memory:")

In [3]:
# create a collection
from qdrant_client.models import Distance, VectorParams

collection_creation = client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

In [4]:
# add vectors
from qdrant_client.http.models import PointStruct

upsert_status = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={"city": ["Berlin", "London"]},
        ),
        PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={"city": ["Berlin", "Moscow"]},
        ),
        PointStruct(
            id=4,
            vector=[0.18, 0.01, 0.85, 0.80],
            payload={"city": ["London", "Moscow"]},
        ),
        PointStruct(id=5, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": ["Mumbai", "Frankfurt"]}),
    ],
)

In [5]:
# run a query
search_result = client.search(
    collection_name="test_collection", query_vector=[0.2, 0.1, 0.9, 0.7], limit=3
)
search_result

[ScoredPoint(id=4, version=0, score=1.362000031210482, payload={'city': ['London', 'Moscow']}, vector=None),
 ScoredPoint(id=1, version=0, score=1.2729999996721744, payload={'city': 'Berlin'}, vector=None),
 ScoredPoint(id=3, version=0, score=1.2080000013113021, payload={'city': ['Berlin', 'Moscow']}, vector=None)]

In [6]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    limit=3,
)
search_result

[ScoredPoint(id=4, version=0, score=1.362000031210482, payload={'city': ['London', 'Moscow']}, vector=None),
 ScoredPoint(id=2, version=0, score=0.8709999993443489, payload={'city': ['Berlin', 'London']}, vector=None)]